# Install dependencies
Run this cell the first time you open this notebook.

In [ ]:
%pip install python-pptx
%pip install lyricsgenius

# Define song name and artist
Change these for each song!

In [ ]:
# Change below lines to whatever the song name and artist are. These should be the only lines you really need to change
SONG_NAME = 'relate'
SONG_ARTIST = 'for king and country'

# Set up webscraper to get lyrics

In [ ]:
from lyricsgenius import Genius
import logging

token = "MKXDngQJSmRPb_5r87eprRrAdAmfg3RUdduCCPQxwOg35j1idDACH3a7Abtyy-ZQ"
%env GENIUS_ACCESS_TOKEN=$token
genius = Genius()

In [ ]:
# artist = genius.search_artist(SONG_ARTIST, max_songs=5)
# logging.debug(artist.songs)
song = genius.search_song(SONG_NAME, SONG_ARTIST)

In [ ]:
song.lyrics

# Define helpful functions

### Parsing Lyrics

In [ ]:
def get_stanza(lyrics):
    # Divide lyrics into lines

    l = len(lyrics)
    # logging.debug(l)
    lines = []
    line = ''
    isValidLine = lambda _line: _line != '' and not line.isdigit() and not "Get tickets as low as" in _line # for filtering out empty lines, headers, and any concert ads (which sometimes appear unexpectedly at the ends of stanzas)

    i = 0
    in_bracket = in_parens = False
    while i < l:
        char = lyrics[i]
        i += 1
        # if i > l: break
        if char == ']':
            # logging.debug('1')
            in_bracket = False
            if char == ']': # stanza complete, next header skipped
                if isValidLine(line): lines.append(line+char)
                line = ''
            continue
        elif char == ')':
            in_parens = False
            continue
        if in_bracket:
            #  logging.debug('2')
            line += char
            continue
        if in_parens:
            continue # ignore parentheticals
        if char in '[':
            # logging.debug('3')
            in_bracket = True
            if isValidLine(line): lines.append(line)
            line = char if char == '[' else '' # entering header
        elif char == '(':
            in_parens = True
            continue
        elif char == '\n':
            # logging.debug('4')
            if line != '' and line[-1] != char:
                # logging.debug('5')
                if isValidLine(line): lines.append(line)
                line = ''
        else:
            # logging.debug(l)
            line += char

    if isValidLine: lines.append(line) # append last line

    # Yield stanzas

    isHeader = lambda text: text[0] == '[' and text[-1] == ']'

    j = 1
    stanza = []
    while j < len(lines):
        clean_line = lines[j].strip()  # remove leading or trailing whitespace

        # remove end-of-line annotations
        last_word = clean_line.split(' ')[-1]
        if last_word[-1].isdigit() and not last_word.isdigit():
            while last_word[-1].isdigit() and not last_word.isdigit():
                clean_line = clean_line[:-1]
                last_word = clean_line.split(' ')[-1]

        if isHeader(clean_line):
            if stanza != []:
                s = stanza[:]
                stanza = []
                yield s # yield next stanza
        else:
            stanza.append(clean_line)
        j += 1
    yield stanza # yield final stanza

Test stanza extraction:

In [ ]:
lyrics = list((song.lyrics.replace('Embed', '')).replace('You might also like', ''))
stanzas = get_stanza(lyrics)

In [ ]:
print(list(stanzas)) #test

### Making presentations

In [ ]:
from pptx import Presentation
from pptx.dml.color import RGBColor
from pptx.util import Inches, Pt
from pptx.enum.text import PP_ALIGN, MSO_AUTO_SIZE, PP_PARAGRAPH_ALIGNMENT

to_rgb = lambda color: RGBColor.from_string(color) if type(color) == str else RGBColor(color[0], color[1], color[2]) # convert hex string to RGBColor

def set_bg(slide, color='000000'):
    "Set the background color of the given slide to the given color."
    fill = slide.background.fill
    fill.solid()
    fill.fore_color.rgb = to_rgb(color)

def add_title_slide(prs, name, artist, color='FFFFFF'):
    "Add a title slide with the given song name and artist."
    title_slide_layout = prs.slide_layouts[0]
    slide = prs.slides.add_slide(title_slide_layout)
    set_bg(slide)
    title = slide.shapes.title
    subtitle = slide.placeholders[1]

    for box in [title, subtitle]:
        left, top, width, height = Inches(8) - box.width//2, box.top, box.width, box.height
        box.left, box.top, box.width, box.height = left, top, width, height

    title.text = name
    title.text_frame.paragraphs[0].font.color.rgb = to_rgb(color)

    subtitle.text = artist


def add_text_slide(prs, text_list, size=40, margin=0.5, color='FFFFFF'):
    "Add a slide with a textbox containing the given text."
    blank_slide_layout = prs.slide_layouts[6]
    slide = prs.slides.add_slide(blank_slide_layout)
    set_bg(slide)

    top = width = height = Inches(margin)
    left = Inches(8) - width//2
    top *= 2
    txBox = slide.shapes.add_textbox(left, top, width, height)
    tf = txBox.text_frame

    tf.text = '\n'.join(text_list)
    p = tf.paragraphs[0]
    p.font.size = Pt(size)
    p.alignment = PP_PARAGRAPH_ALIGNMENT.CENTER
    tf.auto_size = MSO_AUTO_SIZE.TEXT_TO_FIT_SHAPE

    for p in tf.paragraphs:
        p.font.color.rgb = to_rgb(color)
        p.font.size = Pt(size)
        p.alignment = PP_PARAGRAPH_ALIGNMENT.CENTER


# Make the slideshow!

In [ ]:
prs = Presentation()
prs.slide_width, prs.slide_height = Inches(16), Inches(9)

add_title_slide(prs, song.title, song.artist)

# Remove links and annotations from lyrics
lyrics = (song.lyrics.replace('Embed', '')).replace('You might also like', '').split(' ')
for word in lyrics:
    # logging.debug(word)
    if word == '': continue
    if word[-1].isdigit() and not word.isdigit():
        while word[-1].isdigit() and not word.isdigit():
            word = word[:-1]
lyrics = list(' '.join(lyrics))

for stanza in get_stanza(lyrics): # make lyric slides for each song section (maximum 6 lines per slide)
    num_lines = len(stanza)
    if num_lines < 7:
        add_text_slide(prs, stanza)
    else:
        max_lines_per_slide = 4
        if num_lines % max_lines_per_slide != 0:
            for n in (5,3):
                if num_lines % n == 0:
                    max_lines_per_slide = n
                    break
        i = 0
        while i < num_lines:
            text = stanza[i:i+max_lines_per_slide]
            add_text_slide(prs, text)
            i += max_lines_per_slide

prs.save(song.title+'.pptx')
print('Done! Saved to '+song.title+'.pptx')

Finished! The slideshow should be saved to your local computer as "{Song Title}.pptx" in this folder.